In [4]:
import numpy as np
import os
import itertools
import pickle
import random
import sys

import matplotlib.pyplot as plt

from keras.models import model_from_json
from keras import regularizers, optimizers
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
model_json = '/home/roserustowicz/crop-type-mapping/models/best_nn_model.json'
model_hdf5 = '/home/roserustowicz/crop-type-mapping/models/best_nn_model.hdf5'
test_list_pkl = '/home/data/ghana/ghana_full_test'
test_imgs_dir = '/home/data/ghana/s2_npy'

with open(test_list_pkl, 'rb') as f:
    test_grids = pickle.load(f)
    
test_fnames = [os.path.join(test_imgs_dir,f) for f in os.listdir(test_imgs_dir) if f[9:15] in test_grids and f.endswith('.npy')]
test_fnames.sort()

for idx in range(0, len(test_fnames), 2):
    s2_img = np.load(test_fnames[idx])
    clouds = np.load(test_fnames[idx+1])
    
    print(s2_img.shape)
    print(clouds.shape)


(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 49)
(64, 64, 49)
(10, 64, 64, 49)
(64, 64, 49)
(10, 64, 64, 49)
(64, 64, 49)
(10, 64, 64, 49)
(64, 64, 49)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 50)
(64, 64, 50)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 26)
(64, 64, 26)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 64, 51)
(64, 64, 51)
(10, 64, 6

In [3]:
# load json and create model
json_file = open(model_json, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_hdf5)
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

pred = np.argmax(loaded_model.predict(X_train), axis=1)

Loaded model from disk


NameError: name 'X' is not defined